In [2]:
from keras.applications.vgg16 import VGG16
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.utils import np_utils
import numpy as np 
from numpy import genfromtxt

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls "/content/gdrive/My Drive/Datasets/Digit_Recognizer"


test.csv  train.csv


In [0]:
image_size = None

def training_data(file_name):
    raw_data = genfromtxt(file_name, delimiter=',', skip_header=1)

    raw_sample_image = raw_data[0][1:]
    image_size = int(np.sqrt(len(raw_sample_image)))
    
    print(image_size)
    
    X_shape = (len(raw_data), image_size, image_size, 1)
    y_shape = (len(raw_data), 10)

    X_data = np.zeros(X_shape)
    y_data = np.zeros(y_shape)
    
    for index, datum in enumerate(raw_data):
        X_data[index] = np.array(datum[1:]/255).reshape(image_size, image_size, 1)
        y_data[index] = np_utils.to_categorical(int(datum[0]), 10)        

    return X_data, y_data
  
def load_eval_data(file_name):
    raw_data = np.genfromtxt(file_name, delimiter=',', skip_header=1)

    raw_sample_image = raw_data[0]
    image_size = int(np.sqrt(len(raw_sample_image)))
    X_shape = (len(raw_data), image_size, image_size, 1)

    X_data = np.zeros(X_shape)
    for index, datum in enumerate(raw_data):
        X_data[index] = np.array(datum/255).reshape(image_size, image_size, 1)

    return X_data
  

In [7]:
train_file = '/content/gdrive/My Drive/Datasets/Digit_Recognizer/train.csv'
test_file = '/content/gdrive/My Drive/Datasets/Digit_Recognizer/test.csv'

print("Loading training data")
X_train, y_train = training_data(train_file)

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=(image_size, image_size, 1)))
model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Dropout(0.3))
model.add(GlobalAveragePooling2D())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath=model_weights_file, verbose=1, save_best_only=True)
stopper = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=1, mode='auto')
hist = model.fit(X_train, y_train, batch_size=32, epochs=1000, validation_split=0.2, callbacks=[checkpointer, stopper], verbose=1, shuffle=True)


Loading training data
28
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, None, None, 16)    160       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 16)    2320      
_________________________________________________________________
dropout_1 (Dropout)          (None, None, None, 16)    0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, None, None, 16)    0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 32)    4640      
______________________________________________________________

In [0]:
output_file = '/content/gdrive/My Drive/Datasets/Digit_Recognizer/submission.csv'
X_eval = load_eval_data(test_file)

with open(output_file, 'w') as f:
    f.write('ImageId,Label\n')
    y_eval = model.predict(X_eval)
    for index, y_hat in enumerate(y_eval):
        prediction = np.argmax(y_hat)
        f.write(str(index+1)+','+str(prediction)+'\n')
    f.close()